In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import math

np.random.seed(0)

## Generate data, Estimators

In [3]:
# params
p = 1000
c1 = 2
c2 = 2
n1 = c1 * p
n2 = c2 * p
sigma = 0.1

# covariates
X1 = np.random.randn(n1, p) / p
X2 = np.random.randn(n2, p) / p

# models
beta_t = np.random.randn(p, 1) / p

# labels
Y2 = X2 @ beta_t + sigma * np.random.randn(n2, 1)
hat_beta_t = np.linalg.inv(X2.T @ X2) @ X2.T @ Y2

# test data
test_n2 = n2 * 10
test_X2 = np.random.randn(test_n2, p) / p
test_Y2 = test_X2 @ beta_t

## Test errors and transfer

In [4]:
d = 2.0
beta_s = beta_t + d * np.random.randn(p, 1)
Y1 = X1 @ beta_s + sigma * np.random.randn(n1, 1)

hat_beta_st = np.linalg.inv(X1.T @ X1 + X2.T @ X2) @ (X1.T @ Y1 + X2.T @ Y2)

te_beta_t = np.linalg.norm(test_X2 @ hat_beta_t - test_Y2) ** 2 / test_n2
te_beta_st = np.linalg.norm(test_X2 @ hat_beta_st - test_Y2) ** 2 / test_n2

In [9]:
print('Test error:', 'STL', te_beta_t, 'MTL', te_beta_st)
if te_beta_st < te_beta_t:
    print('Positive transfer')
    
delta_beta = d**2 * (c1 + c2)**3 / (c1 + c2 - 1)**3
delta_var = sigma**2 * c1 / ((c2 - 1) * (c1 + c2 - 1))

print(p * (delta_var - c1**2 * delta_beta))

empirical_var = sigma**2 * (np.trace(np.linalg.inv(X2.T @ X2)) - np.trace(np.linalg.inv(X1.T @ X1 + X2.T @ X2))) / p

print('var', empirical_var, 'delta_var', delta_var)

Test error: STL 0.010924393048297737 MTL 0.005054444089174922
Positive transfer
-37919.259259259255
var 6.662935165955606 delta_var 0.006666666666666668
